In [1]:
%load_ext autoreload
%autoreload 2

In [115]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bokeh.io import export_png, output_file, output_notebook
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show
from bokeh.sampledata.autompg2 import autompg2 as df
from bokeh.transform import factor_cmap, jitter

import wandb
from adptSeg.adaptation.model_wrapper import FeatureType, ModelFeaturesExtractor
from fundseg.data.data_factory import ALL_DATASETS
from fundseg.utils.checkpoints import load_model_from_checkpoints

In [3]:
api = wandb.Api()
runs = api.runs("liv4d-polytechnique/Probing-Lesions-Segmentation-Positions")

In [76]:
def get_confMat(df):
    pass


results = []
for r in runs:
    ftype = r.config["feature_type"]
    pos = r.config["position"]
    table = r.logged_artifacts()[0]
    table.download()

    table_name = table.name
    avg_table_path = r.summary["conf_mat_table"]["path"]
    table_path = f"artifacts/{table_name}/conf_mat_table.table.json"
    with open(table_path) as f:
        data = json.load(f)
    df = pd.DataFrame(data["data"], columns=data["columns"])
    datasets = df["Actual"].unique()
    datasets.sort()
    confMat = np.zeros((len(datasets), len(datasets)))
    for i, dataset in enumerate(datasets):
        for j, dataset2 in enumerate(datasets):
            confMat[i, j] = df[(df["Actual"] == dataset) & (df["Predicted"] == dataset2)]["nPredictions"].values[0]

    results.append(
        {
            "feature_type": ftype,
            "position": pos,
            "confMat": confMat,
            "datasets": datasets,
            "n_features": r.config["n_features"],
        }
    )

results.sort(key=lambda x: x["position"])
results.sort(key=lambda x: x["feature_type"], reverse=True)

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [139]:
def get_f1_scores(confMat):
    TP = np.diag(confMat)
    P = np.sum(confMat, axis=1)

    PP = np.sum(confMat, axis=0)
    FP = PP - TP
    FN = np.sum(confMat, axis=1) - TP
    TN = np.sum(confMat) - (TP + FP + FN)
    N = np.sum(confMat) - P
    TPR = TP / P
    TNR = TN / N
    F1 = 2 * TP / (2 * TP + FP + FN)
    return (TPR + TNR) / 2


output_notebook()

classes = [(r["feature_type"], r["position"]) for r in results]
feats = [f"\n(f={r['n_features']})" for r in results]
classes = ["\n Pos: ".join(map(str, c)).capitalize() for c in classes]
classes = [c + f for c, f in zip(classes, feats)]
p = figure(x_range=classes, height=500, width=800, toolbar_location=None)
p.xgrid.grid_line_color = None

scores = [get_f1_scores(r["confMat"]) for r in results]
upper = np.percentile(scores, 75, axis=1)
lower = np.percentile(scores, 25, axis=1)

source = ColumnDataSource(data=dict(base=classes, upper=upper, lower=lower))

error = Whisker(base="base", upper="upper", lower="lower", source=source, level="annotation", line_width=1)
error.upper_head.size = 20
error.lower_head.size = 20


markers = ["circle", "square", "triangle", "hex", "plus"] * len(classes)


df_classes = np.ravel([[c] * len(scores[0]) for c in classes])

legend = sorted(ALL_DATASETS)
legend = [l.name for l in legend] * len(classes)

df_dict = {"classes": df_classes, "scores": np.ravel(scores), "markers": np.ravel(markers), "legend": legend}
df = pd.DataFrame(df_dict)
# p.add_layout(error)

p.scatter(
    jitter("classes", 0.1, range=p.x_range),
    "scores",
    marker="markers",
    source=df,
    alpha=0.5,
    size=13,
    legend_group="legend",
    line_color="white",
    color=factor_cmap("classes", "TolRainbow10", classes),
)
p.line(classes, np.mean(scores, axis=1), line_width=2, color="black")
p.scatter(classes, np.mean(scores, axis=1), line_width=2, color="black")

p.text(
    classes,
    np.mean(scores, axis=1) + 0.02,
    text=[f"{x:.1%}" for x in np.mean(scores, axis=1)],
    text_align="center",
    text_font_size="12px",
)
p.legend.location = "bottom"
p.legend.orientation = "horizontal"
p.toolbar_location = None
# p.background_fill_color = None
p.outline_line_color = None
p.background_fill_alpha = 0
export_png(p, filename="balanced_acc_scores.png")
# show(p)

Loading BokehJS ...

'/home/clement/Documents/Projets/MultiStyle_FundusLesionSegmentation/notebooks/probe_position/balanced_acc_scores.png'